# Fine-tune Llama 3.1 8B with Unsloth

To set it up fill the configuration constants at the beginning of the book layed out in upper case


In [ ]:
!pip install -qqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install -qqq --no-deps {xformers} trl peft accelerate bitsandbytes triton --progress-bar off
!pip install -U "transformers" "huggingface_hub"

import os
import torch
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-1.1.7-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.11.6 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,<=4.57.2,>=4.51.3, but you have transformers 4.57.3 which is incompatible.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make 

In [ ]:
REPO_FINETUNED = ""
REPO_GGUF = ""

## 1. Load model for PEFT

In [ ]:
# Output dir for checkpoints
output_dir = "output_9070xt_checkpoints"
# Load model
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# Prepare model for PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196
None


## 2. Prepare data and tokenizer

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}
)

def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

#10k samples used instead of the 100k the dataset has
dataset = load_dataset("mlabonne/FineTome-100k", split="train[:10000]")
dataset = dataset.map(apply_template, batched=True)

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## 3. Training

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
        # --- CRASH PROTECTION SETTINGS ---
        save_strategy="steps",       # Save by steps, not just epochs
        save_steps=250,              # Save every 250 steps
        save_total_limit=2,          # Keep only the last 2 checkpoints to save disk space
        load_best_model_at_end=False,
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/10000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 625
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WAN

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.290900
2,1.181000
3,1.149700
4,1.146800
5,1.108200
6,0.997700
7,1.037700
8,0.998600
9,0.843800
10,0.904100


Step,Training Loss
1,1.290900
2,1.181000
3,1.149700
4,1.146800
5,1.108200
6,0.997700
7,1.037700
8,0.998600
9,0.843800
10,0.904100


train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇█
train/grad_norm,▇▄█▃▄▂▄▃▃▃▂▂▄▃▂▄▂▄▂▄▂▅▂▃▂▃▃▄▄▃▂▃▁▅▇▃▄▃▃▂
train/learning_rate,▄▇▇█████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▄▃▃▄▄▄▄▂▃▄▂▂▃▃▃▄▃▄▃▁▃▂▂▂▃▃▃▃▂▃▁▁▁▄▂▂▂▄▄
total_flos,3.5251654545024614e+17
train/epoch,1
train/global_step,625
train/grad_norm,0.54402
train/learning_rate,0.0
train/loss,0.6024


TrainOutput(global_step=625, training_loss=0.7317144137382507, metrics={'train_runtime': 8659.2531, 'train_samples_per_second': 1.155, 'train_steps_per_second': 0.072, 'total_flos': 3.5251654545024614e+17, 'train_loss': 0.7317144137382507, 'epoch': 1.0})

### Download the output folder in .zip file

In [ ]:
import shutil
from google.colab import files

shutil.make_archive('output_archive', 'zip', 'output')
files.download('output_archive.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resume from last checkpoint

Run this block only if the finetuning process was halted

In [ ]:
last_checkpoint = None
if os.path.isdir(output_dir):
    checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
    if checkpoints:
        # Sort to find the latest checkpoint number
        checkpoints.sort(key=lambda x: int(x.split('-')[1]))
        last_checkpoint = os.path.join(output_dir, checkpoints[-1])
        print(f"Found checkpoint: {last_checkpoint}. Resuming training...")

trainer.train(resume_from_checkpoint=last_checkpoint)

## 4. Inference test

In [ ]:
# Load model for inference
model = FastLanguageModel.for_inference(model)

messages = [
    {"from": "human", "value": "Is 9.8 larger than 9.9?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

<|im_start|>user
Is 9.8 larger than 9.9?<|im_end|>
<|im_start|>assistant
To determine if 9.8 is larger than 9.9, we can compare their decimal representations. 

First, we convert 9.8 to a decimal number. Since there is no decimal point in 9.8, we can simply add a decimal point after the 8. This gives us 9.8.

Next, we convert 9.9 to a decimal number. Since there is no decimal point in 9.9, we can simply add a decimal point after the 9. This gives us 9.9.

Now we can compare the decimal numbers. 9.8 is less than


## 5. Save trained model

Login to HugginFace

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
model.push_to_hub_merged(REPO_FINETUNED, tokenizer, save_method="merged_16bit")

In [ ]:
tokenizer.save_pretrained("model")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/chat_template.jinja',
 'model/tokenizer.json')

Download model from HF

Only run these 2 blocks if model is not saved in /content

In [ ]:
local_dir = "merged_model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=REPO_FINETUNED,
    max_seq_length=2048,  # Default context length for Llama models
    dtype=None,           # Load the full model for GGUF conversion
    load_in_4bit=False,    # Load the full model for GGUF conversion
)

In [ ]:
model.save_pretrained(local_dir, safe_serialization=True)
tokenizer.save_pretrained(local_dir)

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/chat_template.jinja',
 'merged_model/tokenizer.json')

Transform model to GGUF



In [ ]:
import os
if not os.path.isdir("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git


In [ ]:
quantization_method = "q8_0"

!python llama.cpp/convert_hf_to_gguf.py "./merged_model" \
  --outfile "llama-3.1-8b-finetuned.gguf" \
  --outtype {quantization_method}

INFO:hf-to-gguf:Loading model: merged_model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00004-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00004.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:blk.10.attn_norm.weight,     torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.10.ffn_down.weight,      torch.bfloat16 --> Q8_0, shape = {14336, 4096}
INFO:hf-to-gguf:blk.10.ffn_gate.weight,      torch.bfloat16 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.10.ffn_up.weight,        torch.bfloat16

In [ ]:
from huggingface_hub import HfApi

local_file_path = "llama-3.1-8b-finetuned.gguf"
file_in_repo = "llama-3.1-8b-finetuned.gguf"

api = HfApi()


api.upload_file(
    path_or_fileobj=local_file_path,
    path_in_repo=file_in_repo,
    repo_id=REPO_GGUF,
    commit_message="Add GGUF file"
)